In [64]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/global/cfs/projectdirs/des/zhou/lsst_shear/WeakLensingDeblending')
sys.path.append('/global/cfs/projectdirs/des/zhou/lsst_shear/WeakLensingDeblending/src')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
import numpy as np
from astropy.io import fits
from astropy.table import Table
from tqdm import tqdm,trange
import pickle

In [66]:
from gals_wideblend import *

In [67]:
dc2_path = "../dc2_table_subsampled.fits"
dc2 = Table(fits.open(dc2_path)[1].data)

In [68]:
dc2

ra,dec,g_ab,r_ab,i_ab,z_ab,fluxnorm_bulge,fluxnorm_disk,fluxnorm_agn,pa_disk,pa_bulge,a_d,b_d,a_b,b_b,redshift,galtileid
float64,float64,float32,float32,float32,float32,float32,float32,float32,float64,float64,float32,float32,float32,float32,float64,int64
65.59829063974959,-38.82821040151316,28.042992,28.01334,28.087955,28.174429,0.99921775,0.00078225136,0.0,153.18524924612134,153.18524924612134,0.09199009,0.082533196,0.04396829,0.043571804,2.4828127064539456,8954404156
64.14791809550967,-39.2852577377574,24.077734,24.36398,24.41143,24.466587,0.89238185,0.10761815,0.0,110.23498882067832,110.23498882067832,0.23803718,0.1451242,0.12817658,0.10023341,1.6536637205464282,9756479231
64.94105242487382,-40.6727630199016,29.718473,29.642002,29.415213,29.258001,0.985941,0.014059007,0.0,96.53478172498511,96.53478172498511,0.082211524,0.08023714,0.03620843,0.036022004,1.5094941664259318,9755710733
62.80810917437831,-39.27152354305869,31.442318,31.154013,31.039482,30.825005,0.73025227,0.26974773,0.0,170.96805851528325,170.96805851528325,0.10385448,0.09663466,0.041466024,0.04135432,2.2006777265696327,8889340702
68.76975483273986,-45.13790547500041,33.36518,32.191765,31.794455,31.255127,0.6485413,0.35145873,0.0,160.85517265970728,160.85517265970728,0.034905743,0.030265255,0.019052334,0.018705396,2.0786187763738493,11950640789
67.1522412883226,-35.46663473622269,31.361689,30.940325,30.920937,30.931248,0.76467276,0.23532724,0.0,39.32585006845608,39.32585006845608,0.025811706,0.014040916,0.014595544,0.0139756715,2.9812927100023234,7082834249
68.7299994624655,-43.55074524598453,32.819828,31.916327,30.960804,30.353374,0.624338,0.37566203,0.0,19.729097937659674,19.729097937659674,0.03548009,0.02760433,0.019002393,0.018901002,1.5365271975431192,11318278763
64.64137657157119,-34.479675121924025,29.032701,28.663431,28.6265,28.65434,0.1470591,0.8529409,0.0,70.56253212877999,70.56253212877999,0.09753989,0.046610065,0.04764855,0.04539714,2.712216356477836,6080981738
66.60056536475267,-38.3180210796669,28.087189,28.247736,28.411974,28.535597,0.9987658,0.0012341738,0.0,143.91824503594634,143.91824503594634,0.12085862,0.054017976,0.047378957,0.044943813,1.8853477761584219,8946239183


In [69]:
data = init_wldeblend(survey_bands='lsst-i')

In [86]:
data.noise

420.9240990481278

In [73]:
dc2[i]['galtileid']

8954404156

In [74]:
gal_list = []
for i in trange(len(dc2)):
    gal = get_gal_wldeblend(rng=np.random.RandomState(1234), ind=i, data=data)
    id = dc2[i]['galtileid']
    gal_list.append((gal,id))

100%|██████████| 2017978/2017978 [25:25<00:00, 1322.44it/s] 


In [79]:
with open('../data/gal_list.pkl','wb') as f:
    pickle.dump(gal_list,f)